In [1]:
import os
import boto3
import re
import json
from sagemaker import get_execution_role

role = get_execution_role()

bucket = 'ml-sagemaker-project'# enter your s3 bucket where you will copy data and model artifacts

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac
import urllib.request

# Read the file in chunks

In [3]:
data_2005 = pd.read_csv("https://s3.amazonaws.com/mrinal-ml-sagemaker/Project/2005_data.csv", chunksize = 100000)

In [4]:
with open('2005_codes.json', 'r') as f_open:
    code_maps_2005 = json.load(f_open)

In [5]:
giant_df = pd.concat(data_2005)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (39,40,41,42,43,45,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (39,40,41,42,43,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (39,40,41,42,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (40,41,42,43,44

In [6]:
giant_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2452506 entries, 0 to 2452505
Data columns (total 77 columns):
resident_status                                           int64
education_1989_revision                                   float64
education_2003_revision                                   float64
education_reporting_flag                                  int64
month_of_death                                            int64
sex                                                       object
detail_age_type                                           int64
detail_age                                                int64
age_substitution_flag                                     float64
age_recode_52                                             int64
age_recode_27                                             int64
age_recode_12                                             int64
infant_age_recode_22                                      float64
place_of_death_and_decedents_status           

In [7]:
giant_df.isnull().sum()

resident_status                                                 0
education_1989_revision                                    965336
education_2003_revision                                   1487170
education_reporting_flag                                        0
month_of_death                                                  0
sex                                                             0
detail_age_type                                                 0
detail_age                                                      0
age_substitution_flag                                     2452429
age_recode_52                                                   0
age_recode_27                                                   0
age_recode_12                                                   0
infant_age_recode_22                                      2424003
place_of_death_and_decedents_status                             0
marital_status                                                  0
day_of_wee

In [8]:
with open('2005_codes.json', 'r') as f_open:
    code_maps_2005 = json.load(f_open)

In [9]:
code_maps_2005


{'resident_status': {'1': 'RESIDENTS',
  '2': 'INTRASTATE NONRESIDENTS',
  '3': 'INTERSTATE NONRESIDENTS',
  '4': 'FOREIGN RESIDENTS'},
 'education_1989_revision': {'00': 'No formal education',
  '01-08': 'Years of elementary school',
  '09': '1 year of high school',
  '10': '2 years of high school',
  '11': '3 years of high school',
  '12': '4 years of high school',
  '13': '1 year of college',
  '14': '2 years of college',
  '15': '3 years of college',
  '16': '4 years of college',
  '17': '5 or more years of college',
  '99': 'Not stated'},
 'education_2003_revision': {'1': '8th grade or less',
  '2': '9 - 12th grade, no diploma',
  '3': 'high school graduate or GED completed',
  '4': 'some college credit, but no degree',
  '5': 'Associate degree',
  '6': 'Bachelor’s degree',
  '7': 'Master’s degree',
  '8': 'Doctorate or professional degree',
  '9': 'Unknown'},
 'education_reporting_flag': {'0': '1989 revision of education item on certificate',
  '1': '2003 revision of education it

# Map residents status from json

In [10]:
giant_df['resident_status'] = giant_df['resident_status'].apply(
    lambda x: code_maps_2005['resident_status'][str(x)])

In [11]:
giant_df.head()


,resident_status,education_1989_revision,education_2003_revision,education_reporting_flag,month_of_death,sex,detail_age_type,detail_age,age_substitution_flag,age_recode_52,...,record_condition_18,record_condition_19,record_condition_20,race,bridged_race_flag,race_imputation_flag,race_recode_3,race_recode_5,hispanic_origin,hispanic_originrace_recode
0,RESIDENTS,11.0,NaN,0,1,F,1,45,NaN,35,...,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6
1,RESIDENTS,13.0,NaN,0,1,M,1,61,NaN,38,...,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6
2,RESIDENTS,12.0,NaN,0,1,F,1,79,NaN,41,...,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6
3,RESIDENTS,12.0,NaN,0,1,M,1,50,NaN,36,...,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6
4,RESIDENTS,14.0,NaN,0,1,F,1,68,NaN,39,...,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6


# Modify json for education 1989 revision 

In [12]:
code_maps_2005['education_1989_revision'] = {'0': 'No formal education',
 '1': 'Years of elementary school',
 '2': 'Years of elementary school',
 '3': 'Years of elementary school',
 '4': 'Years of elementary school',
 '5': 'Years of elementary school',
 '6': 'Years of elementary school',
 '7': 'Years of elementary school',
 '8': 'Years of elementary school',
 '9': '1 year of high school',
 '10': '2 years of high school',
 '11': '3 years of high school',
 '12': '4 years of high school',
 '13': '1 year of college',
 '14': '2 years of college',
 '15': '3 years of college',
 '16': '4 years of college',
 '17': '5 or more years of college',
 '99': 'Not stated'}

In [13]:
code_maps_2005

{'resident_status': {'1': 'RESIDENTS',
  '2': 'INTRASTATE NONRESIDENTS',
  '3': 'INTERSTATE NONRESIDENTS',
  '4': 'FOREIGN RESIDENTS'},
 'education_1989_revision': {'0': 'No formal education',
  '1': 'Years of elementary school',
  '2': 'Years of elementary school',
  '3': 'Years of elementary school',
  '4': 'Years of elementary school',
  '5': 'Years of elementary school',
  '6': 'Years of elementary school',
  '7': 'Years of elementary school',
  '8': 'Years of elementary school',
  '9': '1 year of high school',
  '10': '2 years of high school',
  '11': '3 years of high school',
  '12': '4 years of high school',
  '13': '1 year of college',
  '14': '2 years of college',
  '15': '3 years of college',
  '16': '4 years of college',
  '17': '5 or more years of college',
  '99': 'Not stated'},
 'education_2003_revision': {'1': '8th grade or less',
  '2': '9 - 12th grade, no diploma',
  '3': 'high school graduate or GED completed',
  '4': 'some college credit, but no degree',
  '5': 'Asso

# Map edu 1989 revision from json

In [14]:
giant_df['education_1989_revision'] = giant_df['education_1989_revision'].apply(
    lambda x: code_maps_2005['education_1989_revision'][str(int(x))] if pd.notnull(x) else x)

# Map edu 2003 revision from json

In [15]:
giant_df['education_2003_revision'] = giant_df['education_2003_revision'].apply(
    lambda x: code_maps_2005['education_2003_revision'][str(int(x))] if pd.notnull(x) else x)

In [16]:
giant_df['education_2003_revision']

0                                            NaN
1                                            NaN
2                                            NaN
3                                            NaN
4                                            NaN
5                                            NaN
6                                            NaN
7                                            NaN
8                                            NaN
9                                            NaN
10                                           NaN
11                                           NaN
12                                           NaN
13                                           NaN
14                                           NaN
15                                           NaN
16                                           NaN
17                                           NaN
18                                           NaN
19                                           NaN
20                  

# Make new column for edu revision combining columns from both years

In [17]:
giant_df['education_revision'] = giant_df['education_1989_revision']

In [18]:
giant_df.education_revision.fillna(giant_df.education_2003_revision,inplace=True)

# Map detail age type from json

In [19]:
giant_df['detail_age_type'] = giant_df['detail_age_type'].apply(
    lambda x: code_maps_2005['detail_age_type'][str(x)])

In [20]:
giant_df

,resident_status,education_1989_revision,education_2003_revision,education_reporting_flag,month_of_death,sex,detail_age_type,detail_age,age_substitution_flag,age_recode_52,...,record_condition_19,record_condition_20,race,bridged_race_flag,race_imputation_flag,race_recode_3,race_recode_5,hispanic_origin,hispanic_originrace_recode,education_revision
0,RESIDENTS,3 years of high school,NaN,0,1,F,Years,45,NaN,35,...,NaN,NaN,1,NaN,NaN,1,1,100,6,3 years of high school
1,RESIDENTS,1 year of college,NaN,0,1,M,Years,61,NaN,38,...,NaN,NaN,1,NaN,NaN,1,1,100,6,1 year of college
2,RESIDENTS,4 years of high school,NaN,0,1,F,Years,79,NaN,41,...,NaN,NaN,1,NaN,NaN,1,1,100,6,4 years of high school
3,RESIDENTS,4 years of high school,NaN,0,1,M,Years,50,NaN,36,...,NaN,NaN,1,NaN,NaN,1,1,100,6,4 years of high school
4,RESIDENTS,2 years of college,NaN,0,1,F,Years,68,NaN,39,...,NaN,NaN,1,NaN,NaN,1,1,100,6,2 years of college
5,RESIDENTS,Years of elementary school,NaN,0,1,F,Years,89,NaN,43,...,NaN,NaN,3,NaN,NaN,2,3,100,8,Years of elementary school
6,RESIDENTS,4 years of high school,NaN,0,1,F,Years,68,NaN,39,...,NaN,NaN,1,NaN,NaN,1,1,100,6,4 years of high school
7,RESIDENTS,4 years of high school,NaN,0,1,M,Years,61,NaN,38,...,NaN,NaN,3,NaN,NaN,2,3,100,8,4 years of high school
8,RESIDENTS,2 years of college,NaN,0,1,F,Years,73,NaN,40,...,NaN,NaN,1,NaN,NaN,1,1,100,6,2 years of college
9,INTERSTATE NONRESIDENTS,Years of elementary school,NaN,0,1,F,Years,85,NaN,43,...,NaN,NaN,1,NaN,NaN,1,1,100,6,Years of elementary school


In [21]:
giant_df

,resident_status,education_1989_revision,education_2003_revision,education_reporting_flag,month_of_death,sex,detail_age_type,detail_age,age_substitution_flag,age_recode_52,...,record_condition_19,record_condition_20,race,bridged_race_flag,race_imputation_flag,race_recode_3,race_recode_5,hispanic_origin,hispanic_originrace_recode,education_revision
0,RESIDENTS,3 years of high school,NaN,0,1,F,Years,45,NaN,35,...,NaN,NaN,1,NaN,NaN,1,1,100,6,3 years of high school
1,RESIDENTS,1 year of college,NaN,0,1,M,Years,61,NaN,38,...,NaN,NaN,1,NaN,NaN,1,1,100,6,1 year of college
2,RESIDENTS,4 years of high school,NaN,0,1,F,Years,79,NaN,41,...,NaN,NaN,1,NaN,NaN,1,1,100,6,4 years of high school
3,RESIDENTS,4 years of high school,NaN,0,1,M,Years,50,NaN,36,...,NaN,NaN,1,NaN,NaN,1,1,100,6,4 years of high school
4,RESIDENTS,2 years of college,NaN,0,1,F,Years,68,NaN,39,...,NaN,NaN,1,NaN,NaN,1,1,100,6,2 years of college
5,RESIDENTS,Years of elementary school,NaN,0,1,F,Years,89,NaN,43,...,NaN,NaN,3,NaN,NaN,2,3,100,8,Years of elementary school
6,RESIDENTS,4 years of high school,NaN,0,1,F,Years,68,NaN,39,...,NaN,NaN,1,NaN,NaN,1,1,100,6,4 years of high school
7,RESIDENTS,4 years of high school,NaN,0,1,M,Years,61,NaN,38,...,NaN,NaN,3,NaN,NaN,2,3,100,8,4 years of high school
8,RESIDENTS,2 years of college,NaN,0,1,F,Years,73,NaN,40,...,NaN,NaN,1,NaN,NaN,1,1,100,6,2 years of college
9,INTERSTATE NONRESIDENTS,Years of elementary school,NaN,0,1,F,Years,85,NaN,43,...,NaN,NaN,1,NaN,NaN,1,1,100,6,Years of elementary school


In [22]:
set(giant_df['detail_age_type'])

{'Days', 'Hours', 'Minutes', 'Months', 'Years', nan}

# Convert ages to same format

In [23]:
old_giant = giant_df['detail_age_type']

In [24]:
for i in range(0,len(giant_df['detail_age_type'])):
    if giant_df['detail_age_type'][i] == 'Days':
        giant_df['detail_age'][i] = giant_df['detail_age'][i]/365.0
    elif giant_df['detail_age_type'][i] == 'Hours':
        giant_df['detail_age'][i] = giant_df['detail_age'][i]/(365.0*24.0)
    elif giant_df['detail_age_type'][i] == 'Minutes':
        giant_df['detail_age'][i] = giant_df['detail_age'][i]/(365.0*24.0*60.0)
    elif giant_df['detail_age_type'][i] == 'Months':    
        giant_df['detail_age'][i] = giant_df['detail_age'][i]/12.0

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-

In [26]:
giant_df

,resident_status,education_1989_revision,education_2003_revision,education_reporting_flag,month_of_death,sex,detail_age_type,detail_age,age_substitution_flag,age_recode_52,...,record_condition_19,record_condition_20,race,bridged_race_flag,race_imputation_flag,race_recode_3,race_recode_5,hispanic_origin,hispanic_originrace_recode,education_revision
0,RESIDENTS,3 years of high school,NaN,0,1,F,Years,45,NaN,35,...,NaN,NaN,1,NaN,NaN,1,1,100,6,3 years of high school
1,RESIDENTS,1 year of college,NaN,0,1,M,Years,61,NaN,38,...,NaN,NaN,1,NaN,NaN,1,1,100,6,1 year of college
2,RESIDENTS,4 years of high school,NaN,0,1,F,Years,79,NaN,41,...,NaN,NaN,1,NaN,NaN,1,1,100,6,4 years of high school
3,RESIDENTS,4 years of high school,NaN,0,1,M,Years,50,NaN,36,...,NaN,NaN,1,NaN,NaN,1,1,100,6,4 years of high school
4,RESIDENTS,2 years of college,NaN,0,1,F,Years,68,NaN,39,...,NaN,NaN,1,NaN,NaN,1,1,100,6,2 years of college
5,RESIDENTS,Years of elementary school,NaN,0,1,F,Years,89,NaN,43,...,NaN,NaN,3,NaN,NaN,2,3,100,8,Years of elementary school
6,RESIDENTS,4 years of high school,NaN,0,1,F,Years,68,NaN,39,...,NaN,NaN,1,NaN,NaN,1,1,100,6,4 years of high school
7,RESIDENTS,4 years of high school,NaN,0,1,M,Years,61,NaN,38,...,NaN,NaN,3,NaN,NaN,2,3,100,8,4 years of high school
8,RESIDENTS,2 years of college,NaN,0,1,F,Years,73,NaN,40,...,NaN,NaN,1,NaN,NaN,1,1,100,6,2 years of college
9,INTERSTATE NONRESIDENTS,Years of elementary school,NaN,0,1,F,Years,85,NaN,43,...,NaN,NaN,1,NaN,NaN,1,1,100,6,Years of elementary school


In [28]:
code_maps_2005

{'resident_status': {'1': 'RESIDENTS',
  '2': 'INTRASTATE NONRESIDENTS',
  '3': 'INTERSTATE NONRESIDENTS',
  '4': 'FOREIGN RESIDENTS'},
 'education_1989_revision': {'0': 'No formal education',
  '1': 'Years of elementary school',
  '2': 'Years of elementary school',
  '3': 'Years of elementary school',
  '4': 'Years of elementary school',
  '5': 'Years of elementary school',
  '6': 'Years of elementary school',
  '7': 'Years of elementary school',
  '8': 'Years of elementary school',
  '9': '1 year of high school',
  '10': '2 years of high school',
  '11': '3 years of high school',
  '12': '4 years of high school',
  '13': '1 year of college',
  '14': '2 years of college',
  '15': '3 years of college',
  '16': '4 years of college',
  '17': '5 or more years of college',
  '99': 'Not stated'},
 'education_2003_revision': {'1': '8th grade or less',
  '2': '9 - 12th grade, no diploma',
  '3': 'high school graduate or GED completed',
  '4': 'some college credit, but no degree',
  '5': 'Asso

In [29]:
set(giant_df['method_of_disposition'])

{'B', 'C', 'D', 'E', 'O', 'R', 'U'}